# Лабораторна робота 3 з "Асиметричних криптосистем та протоколів"
## Тема: Криптосистема Рабіна. Атака на протокол доведення без розголошення.

**Виконали**\
Бондар Петро, ФІ-03\
Кістаєв Матвій, ФІ-03

In [6]:
import hashlib
import requests
from optimus import *

In [7]:
KEY_LENGTH = 256


class Rabin_Server:
    __base_url = 'http://asymcryptwebservice.appspot.com/rabin/'
    s = requests.Session()
    n = None
    b = None
    
    # Setup server private key and receive server pub key
    def set_server_key(self, key_l: int) -> (str, str):
        req = f'{self.__base_url}serverKey?keySize={key_l}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        self.n, self.b = (int(r['modulus'], 16), int(r['publicExponent'], 16))
        return (self.n, self.b)
    
    # Ask server to encrypt
    def encrypt(self, M: str, rec_n, rec_b, type='TEXT'):
        req = f'{self.__base_url}encrypt?modulus={format(rec_n, "X")}&b={format(rec_b, "X")}&message={M}&type={type}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return (r['cipherText'], r['parity'], r['jacobiSymbol'])
    
    # Ask server to decrypt this message with his private keys
    def decrypt(self, C: str, p: int, j: int, type='TEXT'):
        req = f'{self.__base_url}decrypt?cipherText={C}&expectedType={type}&parity={p}&jacobiSymbol={j}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['message']
    
    # Ask server to sign this message with his private keys
    def sign(self, M: str, type='TEXT'):
        req = f'{self.__base_url}sign?message={M}&type={type}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['signature']
    
    # Verify the message using this public key
    def verify(self, M: str, sign: str, u_n, type='TEXT'):
        req = f'{self.__base_url}verify?message={M}&type={type}&signature={sign}&modulus={format(u_n, "X")}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['verified']


In [8]:
def str2hex(s: str):
    res = ""

    for c in s:
        cb = hex(ord(c))
        res += cb[2::]

    return res

def num2str(n: int):
    text = str()
    while n != 0:
        text += chr(n % 256)
        n //= 256
    
    return text[::-1]

def rabin_format(X: int):
    return 255*(2**(KEY_LENGTH*2 - 16)) + int(str2hex(X), 16) * (2**64) + random.randrange(0, 2**64)

def rabin_unformat(X: int):
    return (X % (pow(2, (KEY_LENGTH*2 - 16)))) // (2**64)

In [16]:
class Rabin_User:
    def __init__(self, p, q, b = 0, serv = Server()):
        print("Initializing user...")
        if not check_prime(p) or not check_prime(q):
            raise RuntimeError("p or q is not a prime number.")
        
        self.serv = serv
        self.p = p
        self.q = q
        self.b = b
        self.n = p*q
        self.get_server_public_key(KEY_LENGTH * 2)

        print(f"User private key (p, q): {(self.d, self.p, self.q)}")
        print(f"User public key (n): {(self.n, self.e)}")
        print(f"User server public key (n): {(self.serv.n, self.serv.e)}")
        print("--------------------------------------------------------")
    

    def get_server_public_key(self, len: int):
        self.serv.set_server_key(len)


    def send_message(self, M: str):
        print("Sending message to the server...")

        M_format = rabin_format(int(str2hex(M), 16))
        
        b1 = M_format % 2
        b2 = jacobi(M_format, self.serv.n)

        C = pow(M_format, 2, self.serv.n)

        M1 = self.serv.decrypt(C, b1, b2)
        
        check = (M1 == M)

        print(f"Sent message: {M}")
        print(f"Sent cyphertext: {C}")
        print(f"Server responce: {M1}")
        
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")


    def send_message_sign(self, M: str):
        print("Sending signature to the server...")
        
        X = rabin_format(int(str2hex(M), 16))
        while not (jacobi(X, self.p) == 1 and jacobi(X, self.q) == 1):
            X = ((X >> 64) << 64) + random.randrange(0, 2**64)

        Roots = sqrt_modpq(X, self.p, self.q)

        S = format(Roots[random.randrange(0, 4)], "X")

        check = self.serv.verify(M, S, self.n)

        print(f"Sent message: {M}")
        print(f"Sent signature: {S}")
        
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")


    def receive_message(self, M: str):
        print("Sending request for message to the server...")
        
        C, b1, b2 = self.serv.encrypt(M, self.n, self.b)
        Roots = sqrt_modpq(int(C, 16), self.p, self.q)

        for m in Roots:
            if [b1, b2] == [m % 2, (jacobi(m, self.p*self.q) == 1)]:
                M1 = m
                break
        
        M2 = (M1 % (pow(2, (KEY_LENGTH*2 - 16)))) // (2**64)
        M2 = num2str(M2)

        check = (M == M2)

        print(f"Sent message: {M}")
        print(f"Received cyphertext: {C}")
        print(f"Decoded cyphertext: {M2}")
        
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")
        
        
    def receive_message_sign(self, M: str):
        print("Sending request for message signature to the server...")
        
        S = self.serv.sign(M)
        M1 = num2str(rabin_unformat(pow(int(S, 16), 2, self.serv.n))) 
        
        check = (M1 == M)

        print(f"Sent message: {M}")
        print(f"Received signature: {S}")
        print(f"Message signed with signature: {M1}")
        
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")
        
        
            


SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (859301257.py, line 61)

True